In [12]:
import sys, os
sys.path.append(os.environ['minotaur'])

import datetime
import seaborn; seaborn.set()


import yaml

%matplotlib inline
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm

from pylab import rcParams
rcParams['figure.figsize'] = 14,3
rcParams['font.family'] = 'Open Sans'

from dbs import redshift
redshift.connect()

from __future__ import division

from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
poly = PolynomialFeatures(degree=2)

In [18]:
#during cleaning and data wrangling process things should be scaled

X = pd.read_csv('final.csv',header=0,index_col=0).copy()
y = X['is_won'].copy()
del X['is_won']



In [26]:

column_lists = [
    list(X.columns),
    [c for c in X.columns if c.startswith('region')],
    [c for c in X.columns if c.startswith('tier')],
    [c for c in X.columns if c.startswith('has')],
    [c for c in X.columns if not c[:3] in ['reg','tie', 'has']]
]
hyperparams = []
for columns in column_lists:
    #training and scoring the model
    #instantiate
    X_subset = X[columns]
    X_subset = pd.DataFrame(poly.fit_transform(X_subset), index=X_subset.index)

    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, random_state = 0)
    

    ridge = Ridge(alpha=15).fit(X_train, y_train)
    y_predtrain = ridge.predict(X_test)
    hyperparams.append([columns, str(ridge), ridge.score(X_test,y_test), ridge.score(X_train,y_train)])

ridge_reg = pd.DataFrame(hyperparams)
ridge_reg.columns = ['columns', 'model', 'test_score', 'train_score']


print('\nAddition of many polynomial features often leads to\n\
overfitting, so we use polynomial featues in combination\n\
with regression that has a regularization penalty, like ridge\n\
regression.\n')
ridge_reg



Addition of many polynomial features often leads to
overfitting, so we use polynomial featues in combination
with regression that has a regularization penalty, like ridge
regression.



,columns,model,test_score,train_score
0,"[call, call_connect, email, marketing_email, s...","Ridge(alpha=15, copy_X=True, fit_intercept=Tru...",0.065529,0.114800
1,"[region_Africa, region_Australia, region_Austr...","Ridge(alpha=15, copy_X=True, fit_intercept=Tru...",0.023516,0.036598
2,"[tier_1, tier_2, tier_3, tier_4, tier_5]","Ridge(alpha=15, copy_X=True, fit_intercept=Tru...",0.000354,0.003029
3,"[has_inbound, has_outbound]","Ridge(alpha=15, copy_X=True, fit_intercept=Tru...",0.017056,0.023180
4,"[call, call_connect, email, marketing_email, s...","Ridge(alpha=15, copy_X=True, fit_intercept=Tru...",0.036819,0.034355


In [ ]:
#chi2 test from sklearn to see which features are overfitting
#checks correlation coef between output variable and train
#won't improve test score
#add loop to run through parameters such as alpha in ridge
#6.5% cvr rate boost

### Lasso Regression

In [7]:
# change tuning param

linlasso = Lasso(alpha=2.0, max_iter = 100000).fit(X_train,y_train)

print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('non zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
      .format(linlasso.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
      .format(linlasso.score(X_test, y_test)))
print('Features with non-zero weight (sorted by abs magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_ )),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

lasso regression linear model intercept: 0.22910850549041503
lasso regression linear model coeff:
[ 0. -0. -0. ...,  0.  0.  0.]
non zero features: 0
R-squared score (training): 0.000
R-squared score (test): -0.001
Features with non-zero weight (sorted by abs magnitude):


In [8]:
#21 because of "biased" intercept
#as the 5 grows the 21 will grow exp so want to be careful of ratio rows to columns
X_poly.shape

(7164, 1378)

In [9]:
X.columns

Index(['call', 'call_connect', 'email', 'marketing_email', 'site_visit',
       '_Africa', '_Australia', '_Australia/New Zealand', '_Belgium',
       '_Benelux', '_Canada', '_Central America', '_Central Europe', '_China',
       '_Eastern Europe', '_Florida', '_France', '_Germany',
       '_Iberian Peninsula', '_India', '_Italy', '_Japan', '_Mexico',
       '_Middle East', '_Northern Europe', '_Russia', '_SE Asia',
       '_Scandinavia', '_South America', '_South Korea',
       '_Southern Europe (Portugal, Spain, Italy, etc)', '_The Netherlands',
       '_US (California)', '_US (Florida)', '_US (Midwest)',
       '_US (New England)', '_US (New York)', '_US (Northwest)', '_US (South)',
       '_US (Southwest)', '_US (Tri-State & Mid-Atlantic)',
       '_US (West Coast & Southwest)', '_United Kingdom',
       '_Western Europe (France and Germany)', '_1', '_2', '_3', '_4', '_5',
       'has_inbound', 'has_outbound'],
      dtype='object')

In [ ]:
#pipeline class from sklearn
#polynomial